In [1]:
!pip install rake-nltk sqlalchemy pymysql pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.5 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()


Saving hashscope-ebf03fb3ec3a.json to hashscope-ebf03fb3ec3a.json


In [3]:
import os

# Ganti 'your-key-file.json' dengan nama file JSON yang diunggah
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "hashscope-ebf03fb3ec3a.json"


In [4]:
!wget https://storage.googleapis.com/cloud-sql-connectors/cloud-sql-proxy/v2.5.0/cloud-sql-proxy.linux.amd64 -O cloud_sql_proxy
!chmod +x cloud_sql_proxy


--2024-12-05 06:26:02--  https://storage.googleapis.com/cloud-sql-connectors/cloud-sql-proxy/v2.5.0/cloud-sql-proxy.linux.amd64
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.217.207, 108.177.11.207, 108.177.12.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.217.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31154279 (30M) [application/octet-stream]
Saving to: ‘cloud_sql_proxy’

cloud_sql_proxy     100%[===================>]  29.71M   128MB/s    in 0.2s    

2024-12-05 06:26:03 (128 MB/s) - ‘cloud_sql_proxy’ saved [31154279/31154279]



In [5]:
!nohup ./cloud_sql_proxy hashscope:asia-southeast2:dbhashscope --port=3306 --credentials-file=hashscope-ebf03fb3ec3a.json > proxy.log 2>&1 &


In [6]:
!cat proxy.log


2024/12/05 06:26:03 Authorizing with the credentials file at "hashscope-ebf03fb3ec3a.json"


In [7]:
from rake_nltk import Rake
import pandas as pd
from sqlalchemy import create_engine

# === KONFIGURASI DATABASE ===
db_user = "root"
db_password = "123456"
db_name = "db_hashscope"
db_ip = "127.0.0.1"
db_port = 3306

# Buat URL koneksi
db_connection_url = f"mysql+pymysql://{db_user}:{db_password}@{db_ip}:{db_port}/{db_name}"

# Buat koneksi SQLAlchemy
engine = create_engine(db_connection_url)



In [12]:
try:
    with connection.cursor() as cursor:
        cursor.execute("SHOW TABLES;")
        tables = cursor.fetchall()
        print("Tabel dalam database:", tables)
except Exception as e:
    print(f"Terjadi kesalahan saat menjalankan query: {e}")


Terjadi kesalahan saat menjalankan query: name 'connection' is not defined


In [9]:
!pip install rake-nltk -q

In [31]:
from rake_nltk import Rake
import pandas as pd

In [34]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [35]:
# Inisialisasi RAKE dengan batas panjang frasa
r = Rake(punctuations=['.', ',', ';', ':', '"', '!', '?', ']', '[', '(', ')', '...', '-'], max_length=7)

In [40]:
# Ambil data dari semua database (x_datasets, web_datasets, youtube_datasets)
# Fungsi untuk mengambil data dari semua database dengan kolom yang benar
def fetch_data_from_all_sources(engine):
    # Ambil data dari x_datasets
    query_x = "SELECT idx_datasets AS id, full_text FROM x_datasets"
    df_x = pd.read_sql(query_x, engine)
    df_x["platform"] = "x_datasets"

    # Ambil data dari web_datasets
    query_web = "SELECT idweb_datasets AS id, full_text FROM web_datasets"
    df_web = pd.read_sql(query_web, engine)
    df_web["platform"] = "web_datasets"

    # Ambil data dari youtube_datasets
    query_youtube = "SELECT idyoutube_datasets AS id, full_text FROM youtube_datasets"
    df_youtube = pd.read_sql(query_youtube, engine)
    df_youtube["platform"] = "youtube_datasets"

    # Gabungkan semua data menjadi satu DataFrame
    all_data = pd.concat([df_x, df_web, df_youtube], ignore_index=True)
    return all_data

data = fetch_data_from_all_sources(engine)


In [41]:
import re

def preprocess_text(text):
    # Hapus mention seperti @username
    text = re.sub(r"@\w+", "", text)
    # Hapus link yang mengandung https atau www
    text = re.sub(r"http\S+|www\.\S+", "", text)
    # Hapus spasi ekstra yang mungkin muncul setelah penghapusan
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [42]:
# Preprocessing teks untuk menghapus kata dengan @
print("Melakukan preprocessing teks...")
data["preprocessed_text"] = data["full_text"].apply(preprocess_text)
print("Preprocessing selesai.")


Melakukan preprocessing teks...
Preprocessing selesai.


In [43]:
def extract_keywords_with_scores(text):
    r.extract_keywords_from_text(text)
    return r.get_ranked_phrases_with_scores()

# Terapkan RAKE ke teks yang sudah diproses
print("Melakukan ekstraksi kata kunci dengan RAKE...")
data["rake_results"] = data["preprocessed_text"].apply(extract_keywords_with_scores)

# Ubah hasil menjadi DataFrame terpisah
rake_results = []
for idx, row in data.iterrows():
    for score, phrase in row["rake_results"]:
        rake_results.append({
            "id": row["id"],
            "score": score,
            "phrase": phrase,
            "platform": row["platform"]
        })

# Konversi hasil RAKE ke DataFrame
rake_df = pd.DataFrame(rake_results)

# Filter hasil berdasarkan skor (> 10)
filtered_rake_df = rake_df[rake_df["score"] > 10]

print("Ekstraksi selesai. Contoh hasil:")
display(filtered_rake_df.head(20))

Melakukan ekstraksi kata kunci dengan RAKE...
Ekstraksi selesai. Contoh hasil:


,id,score,phrase,platform
0,103,49.0,pakai teknologi jaman perang dunia hairan aku,x_datasets
1,105,49.0,kayaknya yg ini sdh pakai teknologi canggih,x_datasets
2,105,36.0,biaya pembuatannya gak masuk mumgkin bro,x_datasets
3,105,25.0,bukan kayak hasil printer warna,x_datasets
4,107,16.0,masuk kuliah wajib militer,x_datasets
5,108,25.0,berarti ini masalah gelombang sinyal,x_datasets
6,109,23.5,d4 supervisor jaminan mutu pangan,x_datasets
7,109,23.5,d4 supervisor jaminan mutu pangan,x_datasets
20,110,49.0,1 dan tingkatkan kualitas grafis game anda,x_datasets
21,110,16.0,# idncitizen # promedia,x_datasets


In [44]:
# Simpan hasil RAKE ke CSV
csv_path = "rake_results.csv"
filtered_rake_df.to_csv(csv_path, index=False)
print(f"Hasil RAKE disimpan sementara ke {csv_path}.")


Hasil RAKE disimpan sementara ke rake_results.csv.


**SQL QUERY UNTUK UPDATE KOLOM FREKUENSI**

UPDATE main_topics
SET frequency = (
    SELECT COUNT(*)
    FROM main_topics AS mt
    WHERE mt.topics_name = main_topics.topics_name
);



In [45]:
def insert_main_topics_to_db_with_platform_check(filtered_rake_df):
    print("Memulai proses insert data ke database...")
    try:
        # Ambil ID yang sudah ada di tabel main_topics
        with engine.connect() as conn:
            existing_ids_query = """
                SELECT x_datasets_idx_datasets, web_datasets_idweb_datasets, youtube_datasets_idyoutube_datasets
                FROM main_topics
            """
            existing_ids = pd.read_sql(existing_ids_query, conn)
            print(f"Jumlah ID yang sudah ada: {len(existing_ids)}")

        # Gabungkan semua ID yang sudah ada
        existing_x_ids = existing_ids["x_datasets_idx_datasets"].dropna().tolist()
        existing_web_ids = existing_ids["web_datasets_idweb_datasets"].dropna().tolist()
        existing_youtube_ids = existing_ids["youtube_datasets_idyoutube_datasets"].dropna().tolist()

        # Filter data baru agar hanya memasukkan ID yang belum ada
        new_data = filtered_rake_df.copy()
        if "platform" not in new_data.columns:
            raise ValueError("Kolom 'platform' harus ada di filtered_rake_df untuk menentukan sumber data.")

        new_data = new_data[
            ~(
                (new_data["platform"] == "x_datasets") & (new_data["id"].isin(existing_x_ids)) |
                (new_data["platform"] == "web_datasets") & (new_data["id"].isin(existing_web_ids)) |
                (new_data["platform"] == "youtube_datasets") & (new_data["id"].isin(existing_youtube_ids))
            )
        ]
        print(f"Jumlah data baru yang akan di-insert: {len(new_data)}")

        # Siapkan data untuk insert
        data_to_insert = []
        for _, row in new_data.iterrows():
            data = {
                "topics_name": row["phrase"],  # Nama topik dari hasil RAKE
                "frequency": 1,               # Frekuensi default adalah 1
                "x_datasets_idx_datasets": None,
                "web_datasets_idweb_datasets": None,
                "youtube_datasets_idyoutube_datasets": None
            }

            # Isi foreign key berdasarkan platform
            if row["platform"] == "x_datasets":
                data["x_datasets_idx_datasets"] = row["id"]
            elif row["platform"] == "web_datasets":
                data["web_datasets_idweb_datasets"] = row["id"]
            elif row["platform"] == "youtube_datasets":
                data["youtube_datasets_idyoutube_datasets"] = row["id"]

            data_to_insert.append(data)

        # Konversi ke DataFrame jika ada data baru
        if data_to_insert:
            df_to_insert = pd.DataFrame(data_to_insert)

            # Insert data baru ke database
            with engine.connect() as conn:
                df_to_insert.to_sql("main_topics", con=conn, if_exists="append", index=False)
            print("Data berhasil disimpan ke database.")
        else:
            print("Tidak ada data baru untuk di-insert.")

    except Exception as e:
        print("Kesalahan saat menyimpan data ke database:")
        print(e)

# Panggil fungsi untuk insert data dengan pengecekan
insert_main_topics_to_db_with_platform_check(filtered_rake_df)


Memulai proses insert data ke database...
Jumlah ID yang sudah ada: 0
Jumlah data baru yang akan di-insert: 242
Data berhasil disimpan ke database.
